In [8]:
# console and directory access
import os
import re
import datetime

# interacting with Amazon AWS
import boto3
from sagemaker.session import Session

# data reading and exporting  
import json
import pandas as pd
import numpy as np

# parsing SEC website for data  
import requests 
from bs4 import BeautifulSoup

## Archived Dealer Data Import from 1993
Parses in dealer information with accompaning CIK code for EDGAR lookup from the SEC dealer registration

In [24]:
def companyName(cik:str) -> str:
    """
    Returns the company name for a given CIK number from the SEC by parsing 
    the Edgar site for current company name filing
    
    Input:
        :param: cik (type str)
            The CIK number for a broker dealer e.g. 887767
    Return:
        :param: (type str)
            Returns the accompanying name with the CIK provided 
            e.g. 1ST GLOBAL CAPITAL CORP. 
    """
    
    # establishing base-url for company name search
    baseURL = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&'
    current_year = datetime.datetime.today().year
    url = baseURL+'CIK={}&type=X-17A-5&dateb={}1231'.format(cik, current_year)
    
    # response time for retrieving company names, returning beautifulsoup obj.
    res = requests.get(url, allow_redirects=True)
    s1 = BeautifulSoup(res.text, 'html.parser')
    
    # select the company information from the SEC website for a particular CIK
    for val in s1.find_all('span', attrs={"class":"companyName"}):
        
        # retrieve the company name from info class
        comp_name = val.text.split('CIK')[0].split('/BD')[0]
        print('\t\tFound CIK {}, company-name {}'.format(cik, comp_name))
        
        return comp_name

In [25]:
def dealerData(years:list, quarters:list=['QTR1', 'QTR2', 'QTR3', 'QTR4'], 
               cik2brokers:dict={'years-covered':[], 'broker-dealers':{}}) -> dict:
    """
    Retrieve dealer data from archived SEC directory, returns a dictionary of essential information
    
    Input:
        :param: years (type list)
            A list of years to check for additional dealer data to be pulled e.g. [1993, 1994, 2000]. NOTE, that
            only the years specified are checked for dealer information. 
        :param: quarters (type list)
            A list of quarters to check for additional dealer data, string must be of the form "QTRX", where X is 
            an integer from 1-4 inclusive default = [QTR1, QTR2, QTR3, QTR4]. 
        :param: cik2brokers (type dictionary)
            A nested dictionary for storing the broker-dealer data as well as the years covered from the archive 
            e.g. {'years-reported':['2020/QTR1', '2020/QTR2'],'broker-dealers':{1904: 'ABRAHAM SECURITIES CORPORATION'}}. 
    Return:
        :param: cik2brokers (type dict)
            Returns a dictionary with CIK:CompanyName relationships e.g. {887767: 1ST GLOBAL CAPITAL CORP.} as well 
            as metadata on how many years and quarters were retrieved last
    """
    
    # archived data website for broker dealer data
    baseURL = 'https://www.sec.gov/Archives/edgar/full-index'
    
    # extract all the years covered from json form (we want to avoid re-runs) 
    archiveDates = ['{}/{}'.format(yt, qt) for yt in years for qt in quarters]
    years_covered = cik2brokers['years-covered']
    
    # itterate through years and quarters for archival search
    for coverage in archiveDates:

        if coverage in years_covered:
            print('\tWe have covered {}'.format(coverage))
            pass

        else:
            searchURL = '{}/{}/form.idx'.format(baseURL, coverage)
            print('\nSearching for broker dealers at {}'.format(searchURL))

            # send request to SEC website to retrieve broker dealer information 
            response = requests.get(searchURL, allow_redirects=True)

            # if reponse type is active we return object with status code 200 (else error)
            if response.status_code == 200:

                # append the coverage year for the cik in question
                cik2brokers['years-covered'].append(coverage)

                # extract only main text from body, selecting terms below dashes '---' 
                # we use triple dashes to avoid improper splits that exist locally with company names
                data = response.text.split('---')[-1]   

                # write contents to a temporary file to read information
                with open('main.txt', 'w') as file: file.write(data)

                # convert text data to dataframe object using a fixed-width-file convention
                df = pd.read_fwf('main.txt', header=None)
                cleanDf = df[~pd.isnull(df[0])]               # strip away rows with NaN from the Form Type column

                # check to see if first column contains information on X-17A-5 filings (use regex for x-17a flag)
                x17_check = cleanDf[0].str.contains('^x-17a', regex=True, flags=re.IGNORECASE)
                x17File = cleanDf[x17_check]

                print('\tFound {} X-17A-5 filings in {}'.format(x17File.shape[0], coverage))

                # check whether X-17A-5 form type was found (if empty pass)
                if not x17File.empty:

                    # CIK number is taken from the last column of the rows splitting url string by row 
                    # e.g. edgar/data/886475/0001019056-10-000046.txt -> 886475
                    last_column = x17File.columns[-1]
                    cikNumbers = x17File[last_column].apply(lambda x: x.split('/')[2]).values
                    
                    # compute dictionary mapping for the CIK and company name for each broker-dealer    
                    dictionary_update = dict(map(lambda x: (x, companyName(x)), cikNumbers))

                    # update the broker-dealer list
                    cik2brokers['broker-dealers'].update(dictionary_update)

                # remove local file after it has been created
                os.remove('main.txt')
        
    return cik2brokers

In [ ]:
if __name__ == "__main__":
    
    # s3 active folder for outputs
    bucket = "ran-s3-systemic-risk"
    folder ="Output/"

    # Amazon Textract client and Sagemaker session
    s3 = boto3.client('s3')
    session = Session()

    paths = session.list_s3_files(bucket, folder)
    
    # determine the interval range for which we look back historically 
    parse_years = np.arange(1993, datetime.datetime.today().year+1)

    # if the CIK-Dealer file is located in the output folder we read it in and pass as argument
    if 'Output/CIKandDealers.json' in paths: 
        # retrieving downloaded files from s3 bucket
        s3.download_file(bucket, 'Output/CIKandDealers.json', 'temp.json')
        
        # read all CIK and Dealer name information from storage
        with open('temp.json', 'r') as f: old_cik2brokers = json.loads(f.read())
            
        cik2brokers = dealerData(years=parse_years, cik2brokers=old_cik2brokers)   
        
        # remove local file after it has been created
        os.remove('temp.json')
    else:
        # otherwise we must run the entire history of broker-dealers registering a X-17A-5 file
        cik2brokers = dealerData(years=parse_years)
        
    # write to a JSON file with accompanying meta information about coverage 
    with open('CIKandDealers.json', 'w') as file:
        json.dump(cik2brokers, file)
        file.close()
    
    # save contents to AWS S3 bucket
    with open('CIKandDealers.json', 'rb') as data:
        s3.upload_fileobj(data, bucket, 'Output/CIKandDealers.json')
        
    # remove local file after it has been created
    os.remove('CIKandDealers.json')
    

	We have covered 1993/QTR1
	We have covered 1993/QTR2
	We have covered 1993/QTR3
	We have covered 1993/QTR4
	We have covered 1994/QTR1
	We have covered 1994/QTR2
	We have covered 1994/QTR3
	We have covered 1994/QTR4
	We have covered 1995/QTR1
	We have covered 1995/QTR2
	We have covered 1995/QTR3
	We have covered 1995/QTR4
	We have covered 1996/QTR1
	We have covered 1996/QTR2
	We have covered 1996/QTR3
	We have covered 1996/QTR4
	We have covered 1997/QTR1
	We have covered 1997/QTR2
	We have covered 1997/QTR3
	We have covered 1997/QTR4
	We have covered 1998/QTR1
	We have covered 1998/QTR2
	We have covered 1998/QTR3
	We have covered 1998/QTR4
	We have covered 1999/QTR1
	We have covered 1999/QTR2
	We have covered 1999/QTR3
	We have covered 1999/QTR4
	We have covered 2000/QTR1
	We have covered 2000/QTR2
	We have covered 2000/QTR3
	We have covered 2000/QTR4

Searching for broker dealers at https://www.sec.gov/Archives/edgar/full-index/2001/QTR1/form.idx
	Found 2 X-17A-5 filings in 2001/QTR1


		Found CIK 1071638, company-name AURORA SECURITIES INC 
		Found CIK 1084325, company-name ALBANY SECURITIES, LLC. 
		Found CIK 715628, company-name STATESIDE SECURITIES, LLC 
		Found CIK 922070, company-name ALERUS SECURITIES CORPORATION 
		Found CIK 1055902, company-name ALETHEIA SECURITIES, INC., 
		Found CIK 867629, company-name ALEX MOORE & COMPANY, INC. 
		Found CIK 1005744, company-name ALEXANDER CAPITAL, L.P. 
		Found CIK 757015, company-name DON ALEXANDER INVESTMENTS, INC. 
		Found CIK 842984, company-name ALEXANDER DUNHAM SECURITIES, INC. 
		Found CIK 842986, company-name ALEXANDER GABOR & CO, INC. 
		Found CIK 911591, company-name ALFS, INC. 
		Found CIK 3588, company-name ALL FUNDS, INC. 
		Found CIK 701520, company-name ALL-VEST SECURITIES, INC. 
		Found CIK 892896, company-name ALLAPREE SECURITIES, INC. 
		Found CIK 1127577, company-name ALLEGIANCE CAPITAL, L.L.C. 
		Found CIK 3683, company-name ALLEN & COMPANY LLC 
		Found CIK 200014, company-name ALLEN & COMPANY OF FLOR

		Found CIK 1054453, company-name ARAPAHO PARTNERS, LLC 
		Found CIK 824662, company-name ARBOR RESEARCH & TRADING, LLC 
		Found CIK 914855, company-name ARBORHEDGE INVESTMENTS, INC. 
		Found CIK 932530, company-name ARCADIA INVESTMENT CORPORATION 
		Found CIK 1053731, company-name ARCADIA SECURITIES, LLC 
		Found CIK 1038165, company-name ARCHELON LLC 
		Found CIK 1071637, company-name ARCHON SECURITIES, LLC 
		Found CIK 1139505, company-name ARCLIGHT FINANCIAL LLC                                  
		Found CIK 1005814, company-name ARGENT CLASSIC CONVERTIBLE ARBITRAGE FUND L.P. 
		Found CIK 857155, company-name ARGENT INVESTMENT SERVICES, INC. 
		Found CIK 215131, company-name ARGUS SECURITIES, INC. 
		Found CIK 944360, company-name ARIEL DISTRIBUTORS, LLC 
		Found CIK 1138684, company-name ARISTATRADE SECURITIES, LLC 
		Found CIK 1041846, company-name ARISTEIA TRADING, L.L.C. 
		Found CIK 1038019, company-name ARIZONA CAPITAL GROUP, INC. 
		Found CIK 789639, company-name ARK FUNDS DI

		Found CIK 832719, company-name BANYAN SECURITIES, LLC 
		Found CIK 9874, company-name BARCLAY INVESTMENTS LLC 
		Found CIK 851376, company-name BARCLAYS CAPITAL INC. 
		Found CIK 1002197, company-name BLACKROCK EXECUTION SERVICES 
		Found CIK 882676, company-name BARING INVESTMENT SERVICES, INC. 
		Found CIK 880915, company-name BARINGTON CAPITAL GROUP, L.P. 
		Found CIK 887602, company-name BARRETT & COMPANY, INC. 
		Found CIK 1087945, company-name BARRETT SECURITIES INC. 
		Found CIK 1001371, company-name BARRIGER & BARRIGER INCORPORATED 
		Found CIK 857294, company-name BARRINGTON ASSOCIATES 
		Found CIK 887084, company-name NANCY BARRON & ASSOCIATES, INC. 
		Found CIK 1085799, company-name RIVIERE SECURITIES LTD. 
		Found CIK 802617, company-name BARTON HALL SECURITIES, INC. 
		Found CIK 230056, company-name BARWELL SECURITIES CORPORATION 
		Found CIK 10283, company-name BASIL INVESTMENT CORPORATION 
		Found CIK 1043117, company-name BASIS FINANCIAL, LLC 
		Found CIK 1013542, com

		Found CIK 753835, company-name BNP PARIBAS SECURITIES CORP. 
		Found CIK 796559, company-name BNY ASSOCIATES INCORPORATED 
		Found CIK 755925, company-name BNY CLEARING SERVICES LLC 
		Found CIK 917380, company-name COWEN EXECUTION SERVICES LLC 
		Found CIK 1088945, company-name BNY INVESTMENT CENTER INC. 
		Found CIK 1018264, company-name BNY TRADING LLC 
		Found CIK 12933, company-name BOENNING & SCATTERGOOD, INC. 
		Found CIK 700332, company-name BOLTON FINANCIAL SERVICES, LLC 
		Found CIK 1119810, company-name BOND EXPRESS, INC. 
		Found CIK 1103218, company-name TRADEWEB DIRECT LLC 
		Found CIK 1113381, company-name BONDHUB, INC. 
		Found CIK 1095240, company-name BONDNET BROKERS INC                                     
		Found CIK 1137614, company-name BONDS DIRECT SECURITIES LLC 
		Found CIK 1140711, company-name BONDWAVE LLC 
		Found CIK 1041759, company-name BOOMER TRADING, LLC 
		Found CIK 830005, company-name EQUINOX GROUP DISTRIBUTORS, LLC 
		Found CIK 701087, company-nam

...

		Found CIK 862451, company-name R.W.PRESSPRICH & CO. 
		Found CIK 865493, company-name R.A. MACKIE & CO., L.P. 
		Found CIK 741286, company-name R.G. FREEMAN SECURITIES, INC. 
		Found CIK 1137530, company-name R.J. MURPHY & ASSOCIATES, LLC 
		Found CIK 1202603, company-name R.J. RUDDEN FINANCIAL, LLC 
		Found CIK 1093442, company-name R.J. THOMPSON SECURITIES, INC. 
		Found CIK 729162, company-name R.M. DUNCAN SECURITIES, INC. 
		Found CIK 818652, company-name R.M.N. SECURITIES, INC. 
		Found CIK 1091146, company-name R.T. HICKMAN SECURITIES, LLC 
		Found CIK 1182014, company-name RABO SECURITIES USA, INC. 
		Found CIK 786464, company-name RAGEN MACKENZIE INVESTMENT SERVICES, LLC 
		Found CIK 939648, company-name WOODSTOCK FINANCIAL GROUP, INC. 
		Found CIK 934263, company-name RAMAT SECURITIES LTD. 
		Found CIK 1014230, company-name RAMIUS SECURITIES LLC 
		Found CIK 1038997, company-name RAMPART FINANCIAL SERVICES, INC. 
		Found CIK 742327, company-name RANCE KING SECURITIES CORP. 


		Found CIK 311996, company-name SADDLEBROOK INVESTMENTS, INC. 
		Found CIK 1101881, company-name SAEN OPTIONS USA INC. 
		Found CIK 1116645, company-name SAFDIE INVESTMENT SERVICES CORP. 
		Found CIK 1146281, company-name SAFIE HOLDINGS LLC 
		Found CIK 919454, company-name SAFIR SECURITIES, INC. 
		Found CIK 1146129, company-name SAGE FINANCIAL, LLC 
		Found CIK 1122461, company-name SAGE PARTNERS SECURITIES, LLC 
		Found CIK 1017038, company-name SAGE SECURITIES CORP. 
		Found CIK 86172, company-name SAGE, RUTTY & CO., INC. 
		Found CIK 1146241, company-name SAINT SOTER CAPITAL, LLC 
		Found CIK 1146196, company-name SAINTS ADVISORS LLC 
		Found CIK 841664, company-name SAL. OPPENHEIM JR. & CIE. SECURITIES INC. 
		Found CIK 1044205, company-name LUPO SECURITIES, LLC 
		Found CIK 1001005, company-name SALOMON REINVESTMENT COMPANY INC 
		Found CIK 1144478, company-name SALT CREEK SECURITIES, LLC 
		Found CIK 310039, company-name SALVATORE & CO., INC. 
		Found CIK 885492, company-name 

		Found CIK 1256072, company-name SL DISTRIBUTORS, INC. 
		Found CIK 757482, company-name SLATER FINANCIAL CORPORATION 
		Found CIK 836464, company-name SLAVIC INVESTMENT CORPORATION 
		Found CIK 1146216, company-name SLGG FINANCIAL AND INSURANCE SERVICES, LLC 
		Found CIK 1078015, company-name SLK-HULL DERIVATIVES LLC 
		Found CIK 792440, company-name SLOAN SECURITIES CORP. 
		Found CIK 1088405, company-name A.J. SLOANE & COMPANY, LLC 
		Found CIK 1193821, company-name TZERO ATS, LLC 
		Found CIK 875890, company-name SMBC NIKKO SECURITIES AMERICA, INC. 
		Found CIK 1173385, company-name SMC OPTION MANAGEMENT, LLC 
		Found CIK 1038027, company-name SMITH ASSET MANAGEMENT, INC. 
		Found CIK 936862, company-name OWL CREEK ASSET MANAGEMENT MARKETING, LLC 
		Found CIK 14632, company-name SMITH, BROWN & GROOVER, INC. 
		Found CIK 790853, company-name SOCHET & COMPANY, INC. 
		Found CIK 91595, company-name SODERBERG DOUGLAS GUNARD                                
		Found CIK 1211327, company-

		Found CIK 849832, company-name STURDIVANT & CO. INC. 
		Found CIK 1173208, company-name SUBURBAN FINANCIAL SERVICES, LLC 
		Found CIK 1070299, company-name SUCCESS TRADE SECURITIES, INC. 
		Found CIK 919442, company-name SUMA SECURITIES, LLC 
		Found CIK 910800, company-name SUMMIT BROKERAGE SERVICES, INC. 
		Found CIK 1053138, company-name LANDMARK INVESTMENT GROUP, INC 
		Found CIK 500734, company-name SUMMIT EQUITIES, INC. 
		Found CIK 1068125, company-name SUMNER HARRINGTON LTD. 
		Found CIK 885752, company-name SUN VALLEY GOLD TRADING, INC. 
		Found CIK 1026770, company-name SUNBELT SECURITIES, INC. 
		Found CIK 1103008, company-name BLACKTORCH SECURITIES, LLC 
		Found CIK 1046242, company-name SUNGARD GLOBAL EXECUTION SERVICES LLC 
		Found CIK 350951, company-name SUNGARD INSTITUTIONAL BROKERAGE INC. 
		Found CIK 883848, company-name SUNRISE SECURITIES CORP. 
		Found CIK 95453, company-name SUNSET FINANCIAL SERVICES, INC. 
		Found CIK 1041849, company-name SUNSTATE EQUITY TRADI

		Found CIK 861354, company-name THE ROCKWELL FINANCIAL GROUP, INC. 
		Found CIK 1146248, company-name THE SECURITIES GROUP, LLC 
		Found CIK 915629, company-name THE SHEMANO GROUP, INC. 
		Found CIK 1170049, company-name THE SONTERRA GROUP, INC. 
		Found CIK 1207245, company-name THE SPARTAN GROUP 
		Found CIK 911028, company-name THE STRATEGIC ALLIANCE CORPORATION 
		Found CIK 1225151, company-name THE STRATEGIC FINANCIAL ALLIANCE, INC. 
		Found CIK 1172572, company-name THE STREET, INC. 
		Found CIK 779484, company-name THE STURGES COMPANY 
		Found CIK 849834, company-name THE TAVENNER COMPANY 
		Found CIK 1209355, company-name THE TIBERIUS QUALIFIED MASTER FUND LTD. 
		Found CIK 1003676, company-name THE TIDAL GROUP, INC. 
		Found CIK 847590, company-name THE TRANSPORTATION GROUP (SECURITIES) LIMITED 
		Found CIK 1121826, company-name THE VERTICAL TRADING GROUP, LLC 
		Found CIK 914866, company-name THE WILLIAMS CAPITAL GROUP, L.P. 
